<h1 style="text-align:center;font-size:40px;">Tools 1 Final Project</h1>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<p style="text-align:center;">Anthony Molieri<br>
Leo Walker</p>
<br>
<br>
<br>
<br>

# Dataset and Motivation

The data used for these analyses were scraped from Indeed and Glassdoor. The data from Indeed contains job descriptions for open Data Scientist positions. The Glassdoor data includes employee generated ratings on benefits and perks for many of the companies present in the Indeed dataset. The datasets were then combined in order to give an idea of how employees rank the benefits at these companies.

*The Indeed data was scraped over ... pulling job descriptions for Data Scientists in 25 cities throughout the US. **Leo** Add Correct Details, here about the Indded data and scraper*

The Glassdoor scraper was designed to pull company names from the the job description dataset. It then looped over the list of employers, searched, and scraped ratings from the benefits page for companies that were found. This data was then added to it's own table in the same database. The scraper was designed to be run multiple times, only searching for benefits ratings that were not already present in the database. 

# Task Definition and Research Question
 
The purpose of this project was to help give a better idea of how employees at companies with open positions rate their benefits. This can help give insight into how desirable a company is as an employer.

# Literature Review

*Add -- I'm not really sure what to add here. Maybe something about turnover in data science? Companies not having great benefits?*

# Data Cleaning

In [7]:
# imports
import numpy as np
import pandas as pd
import sqlite3

In [112]:
# Initialize both tables
cnct = sqlite3.connect('Data/jobs')

indeed_df = pd.read_sql_query('SELECT * FROM indeed_jobs;', cnct)

glassdoor_df = pd.read_sql_query('SELECT companies, type, rating, count_of_ratings FROM ratings;', cnct)

## Cleaning Indeed Data

In [228]:
# # Pull missing salary data from description
# salaryFix = indeed_df[['job_desc', 'est_salary']].copy()

# salaryFix['check1'] = salaryFix['job_desc'].str.contains("\$")
# salaryFix['check2'] = salaryFix['job_desc'].str.contains("salary")

# I'm not great at regex but let's try this
# Only look for salary data when it's missint in 'est_salary' 
indeed_df['salary_extract'] = np.where(indeed_df['est_salary'] == "No Estimated Salary",indeed_df['job_desc'].str.extract("((\$)(([0-9])([0-9])([0-9])(,)([0-9])([0-9])([0-9])|([0-9])([0-9])(,)([0-9])([0-9])([0-9]))( to |-| - )(\$|)(([0-9])([0-9])([0-9])(,)([0-9])([0-9])([0-9])|([0-9])([0-9])(,)([0-9])([0-9])([0-9]))|(\$)([0-9][0-9][0-9],[0-9][0-9][0-9])|(\$)([0-9][0-9],[0-9][0-9][0-9]))")[0], np.NaN)

In [229]:

indeed_df.head()

,index,job_title,company_name,company_location,est salary,job_href,job_desc,est_salary,retrieve_date,salary_extract
0,0,Data Scientist,Visa,"Atlanta, GA+1 location",None,https://www.indeed.com/rc/clk?jk=160fbb9ff7748...,Company Description Visa is a world leader in ...,No Estimated Salary,2022-02-06,NaN
1,1,Data Scientist and Analytics Specialist,Delta,"Atlanta, GA+4 locations",None,https://www.indeed.com/rc/clk?jk=9b1cc6670b2a1...,"United States, Georgia, Atlanta Product Manage...",No Estimated Salary,2022-02-06,NaN
2,2,Data Scientist,Morgan Stanley,"Alpharetta, GA+1 location",None,https://www.indeed.com/rc/clk?jk=09cd279af0796...,Data Scientist Job Number: 3199911 POSTING DA...,No Estimated Salary,2022-02-06,NaN
3,3,"Data Scientist, Mid",Booz Allen Hamilton,"Atlanta, GA 30303 (Downtown area)",None,https://www.indeed.com/rc/clk?jk=94388bcf3050b...,Key Role: Apply broad understanding of the pri...,No Estimated Salary,2022-02-06,NaN
4,4,Data Scientist,JBT,"Alpharetta, GA",None,https://www.indeed.com/rc/clk?jk=c198ebe89e917...,Overview Data Scientist We are a global leadin...,No Estimated Salary,2022-02-06,NaN


## Cleaning Glassdoor Data

In [113]:
# Restructure glassdoor data

# split type and sub-type
glassdoor_df[['new_type', 'sub_type']] = glassdoor_df['type'].str.split(' - ', expand=True)
glassdoor_df.drop(['type'], axis=1, inplace=True)

glassdoor_df.head()

,companies,rating,count_of_ratings,new_type,sub_type
0,Visa,3.7,157,"Insurance, Health & Wellness",Health Insurance
1,Visa,3.6,21,"Insurance, Health & Wellness",Dental Insurance
2,Visa,3.6,18,"Insurance, Health & Wellness",Vision Insurance
3,Visa,3.4,13,"Insurance, Health & Wellness",Life Insurance
4,Visa,3.0,7,"Insurance, Health & Wellness",Health Savings Account (HSA)


In [148]:
# Get rid of text from ratings counts. The original parser wasn't made to handle language other than English
glassdoor_df['count_of_ratings'] = glassdoor_df['count_of_ratings'].str.extract('(\d+)', expand=False)

In [157]:
# get overall benefits scores
scaleScores = glassdoor_df.groupby(['companies', 'new_type']).apply(lambda x: (x["rating"].astype(float)* x["count_of_ratings"].astype(int)).sum() / x["count_of_ratings"].astype(int).sum())

companies        new_type                    
-Social Native-  Insurance, Health & Wellness    1.000000
                 Perks & Discounts               5.000000
1-800 Contacts   Family & Parenting              3.487500
                 Financial & Retirement          3.800000
                 Insurance, Health & Wellness    3.463636
                                                   ...   
us issworld      Vacation & Time Off             2.746667
wpromote         Financial & Retirement          3.833333
                 Insurance, Health & Wellness    4.240000
                 Perks & Discounts               3.500000
                 Vacation & Time Off             4.533333
Length: 10584, dtype: float64

In [147]:
# Turn the calculated series into a dataframe that can be analyzed and merged
scaleScore_temp = scaleScores.to_frame().reset_index()

scaleScore_df = scaleScore_temp.pivot_table(0, ['companies'], 'new_type')

scaleScore_df.reset_index( drop=False, inplace=True ) 
scaleScore_df.reindex(['companies', 'Family & Parenting', 'Financial & Retirement', 'Insurance, Health & Wellness', 'Perks & Discounts', 'Professional Support', 'Vacation & Time Off'], axis=1)

scaleScore_df

new_type,companies,Family & Parenting,Financial & Retirement,"Insurance, Health & Wellness",Perks & Discounts,Professional Support,Vacation & Time Off
0,-Social Native-,NaN,NaN,1.000000,5.000000,NaN,NaN
1,1-800 Contacts,3.4875,3.800000,3.463636,4.140000,4.333333,3.363636
2,1199SEIU Benefit and Pension Funds,2.7000,4.150000,2.857143,2.200000,2.000000,2.500000
3,1upHealth,4.5000,2.000000,3.000000,3.000000,NaN,4.500000
4,22squared,3.8000,2.725000,2.927273,3.333333,NaN,3.777778
...,...,...,...,...,...,...,...
2343,pureIntegration,NaN,NaN,4.500000,NaN,NaN,NaN
2344,the NBA,3.7500,3.800000,3.125000,3.484615,3.333333,2.400000
2345,tunein,4.0000,NaN,3.000000,4.000000,NaN,NaN
2346,us issworld,2.0000,2.500000,2.707692,NaN,1.000000,2.746667


In [155]:
# Calculate a mean of all 6 categories
scaleScore_df['overall_mean'] = scaleScore_df[['Family & Parenting', 'Financial & Retirement', 'Insurance, Health & Wellness', 'Perks & Discounts', 'Professional Support', 'Vacation & Time Off']].mean(axis=1)

# set index to be the company name
scaleScore_df.set_index('companies', inplace=True)

scaleScore_df.head(5)

new_type,Family & Parenting,Financial & Retirement,"Insurance, Health & Wellness",Perks & Discounts,Professional Support,Vacation & Time Off,overall_mean
companies,,,,,,,
-Social Native-,NaN,NaN,1.000000,5.000000,NaN,NaN,3.000000
1-800 Contacts,3.4875,3.800,3.463636,4.140000,4.333333,3.363636,3.764684
1199SEIU Benefit and Pension Funds,2.7000,4.150,2.857143,2.200000,2.000000,2.500000,2.734524
1upHealth,4.5000,2.000,3.000000,3.000000,NaN,4.500000,3.400000
22squared,3.8000,2.725,2.927273,3.333333,NaN,3.777778,3.312677


# Visualizations